In [1]:
import numpy as np 
import open3d as o3d 
import mrob 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys
import pathlib
import copy

sys.path.append("..")

In [4]:
import voxel_slam

In [5]:
def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [6]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"
imu_path = "/home/ach/Desktop/datasets/hilti/hilti_imu.txt"

In [7]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)
ts_to_imu = parse_trajectories_float(imu_path, ts_multiplier=ts_multiplier)

In [8]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=320, number_of_clouds=5, center_distance_threshold=3)

In [9]:
voxel_map = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=2.0)
feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

In [10]:
len(feature_map)

84

In [11]:
voxel_slam.EmptyVoxelsFilter(min_voxel_poses=len(poses)).filter(feature_map)

In [12]:
len(feature_map)

48

In [13]:
def get_inconsistent_voxels(feature_map):
    inconsistent_voxels = []

    for voxel_id, pose_to_points in feature_map.items():
        normals = []
        for pose_id, feature_points in pose_to_points.items():
            normals.append(feature_points.get_plane_equation()[:-1])
        
        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=0.2,
            metric="cosine",
            linkage="single",
            compute_distances=True,
        ).fit(np.asarray(normals))

        if clustering.n_clusters_ > 1:
            inconsistent_voxels.append(voxel_id)

    return inconsistent_voxels

In [14]:
inconsistent_voxels = get_inconsistent_voxels(feature_map)

In [15]:
print("Total number of voxels:", len(feature_map))
print("Number of inconsistent voxels:", len(inconsistent_voxels))

Total number of voxels: 48
Number of inconsistent voxels: 8


In [16]:
def get_bounding_box(voxel_center, voxel_size):
    bounds = []
    a = voxel_size / 2
    for x in range(2):
        for y in range(2):
            for z in range(2):
                b_box = (voxel_center[0] + a * (-1 if x == 0 else 1),
                         voxel_center[1] + a * (-1 if y == 0 else 1),
                         voxel_center[2] + a * (-1 if z == 0 else 1))
                bounds.append(b_box)

    return bounds

In [17]:
def get_box_centroid(bounding_box: np.ndarray):
    return np.apply_along_axis(lambda x: (min(x) + max(x)) / 2, 0, bounding_box)

In [18]:
def point_is_in_box(point, bounding_box):
    bounding_box = np.asarray(bounding_box)
    is_in_box = True 
    for i in range(3):
        is_in_box &= min(bounding_box[:, i]) <= point[i] <= max(bounding_box[:, i]) 

    return is_in_box

In [19]:
def break_inconsistent(voxel_map: voxel_slam.VoxelFeatureMap, 
                       inconsistent_voxels: list[voxel_slam.VoxelKey], 
                       voxel_size_threshold):
    voxel_to_pose_points_map = voxel_map._voxel_to_pose_points_map
    found_inconsistent = False
    for voxel_key in inconsistent_voxels:
        if voxel_key.size <= voxel_size_threshold:
            continue
        
        found_inconsistent = True 
        octant_size = voxel_key.size / 2
        octant_centers = get_bounding_box(voxel_center=voxel_key.centroid, voxel_size=octant_size)
        # Add octant-voxels to voxel_map 
        for oct_center in octant_centers:
            octant_key = voxel_slam.VoxelKey(oct_center, octant_size)
            if octant_key not in voxel_to_pose_points_map:
                voxel_to_pose_points_map[octant_key] = {}
        
        # Assign points to octants
        for pose_id, voxel_points in voxel_to_pose_points_map[voxel_key].items():
            for point, point_id in zip(voxel_points.points, voxel_points.pcd_idx):
                # Find point's octant
                for oct_center in octant_centers:
                    octant_key = voxel_slam.VoxelKey(oct_center, octant_size)
                    if point_is_in_box(point, bounding_box=get_bounding_box(oct_center, octant_size)):
                        octo_points: voxel_slam.Voxel = voxel_to_pose_points_map[octant_key].get(pose_id, voxel_slam.VoxelPoints([], []))
                        octo_points.add_point(point, point_id)
                        voxel_to_pose_points_map[octant_key].update({pose_id: octo_points})

        # Pop old voxel center 
        voxel_to_pose_points_map.pop(voxel_key)
    return found_inconsistent
            

In [20]:
print("Voxel Map size:", len(voxel_map._voxel_to_pose_points_map))

Voxel Map size: 84


In [21]:
break_inconsistent(voxel_map, inconsistent_voxels, voxel_size_threshold=1.0)

True

In [22]:
print("Voxel Map size:", len(voxel_map._voxel_to_pose_points_map))
print("Inconsistent size", len(inconsistent_voxels))

Voxel Map size: 140
Inconsistent size 8


In [23]:
octo_feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

In [24]:
print("Length octo feature map", len(octo_feature_map))

Length octo feature map 140


In [25]:
voxel_slam.EmptyVoxelsFilter(min_voxel_poses=len(poses)).filter(octo_feature_map)

In [26]:
print("Length octo feature map", len(octo_feature_map))

Length octo feature map 43


In [27]:
colored_clouds, color_to_voxel = voxel_map.get_colored_feature_clouds(octo_feature_map)

In [28]:
# o3d.visualization.draw_geometries([colored_clouds[0]])

In [29]:
octo_inconsistent = get_inconsistent_voxels(octo_feature_map)

In [30]:
len(octo_inconsistent)

2

## Test optimization

In [43]:
def break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_threshold=0.25):
    transformed_clouds = [None for _ in range(len(poses))]
    for i in range(len(poses)):
        transformed_clouds[i] = copy.deepcopy(clouds[i]).transform(poses[i])

    optimized_submaps = []
    for i in range(0, len(poses), minimap_size):
        voxel_map = voxel_slam.VoxelFeatureMap(
            transformed_clouds[i:i+minimap_size],
            [np.eye(4) for _ in range(minimap_size)],
            voxel_size=2.0
        )
        feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.01)

        has_inconsistent = True
        adaptive_iterations = 0
        while has_inconsistent:
            adaptive_iterations += 1
            voxel_slam.EmptyVoxelsFilter(min_voxel_poses=minimap_size).filter(feature_map)
            inconsistent_voxels = get_inconsistent_voxels(feature_map)
            has_inconsistent = break_inconsistent(voxel_map, inconsistent_voxels, adaptive_voxelisation_threshold)
            if has_inconsistent:
                feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.003)

        print(f"Submap {i}-{i+minimap_size}:", end='\n')
        print("adaptive_iterations =", adaptive_iterations, end='\n')
        opt_poses, is_converged, chi2 = voxel_slam.BaregBackend(feature_map, minimap_size).get_optimized_poses(1000, verbose=True)
        print("====")

        optimized_submaps.append(
            voxel_slam.aggregate_map(voxel_map.get_colored_feature_clouds(feature_map)[0], opt_poses)
        )

    aggregate_filter = voxel_slam.EmptyVoxelsFilter(min_voxel_poses=2)
    aggregate_filter.set_next(voxel_slam.NormalsFilter(0.2))

    print("Aggregated map:", end=' ')
    aggregate_pipeline = voxel_slam.VoxelSLAMPipeline(
        feature_filter=aggregate_filter,
        optimization_backend=voxel_slam.BaregBackend,
        config=voxel_slam.PipelineConfig(voxel_size=2.0, 
                                         ransac_distance_threshold=0.02, 
                                         filter_cosine_distance_threshold=0.2,
                                         backend_verbose=True)
    )
    o3d.visualization.draw_geometries(optimized_submaps)
    aggregate_output = aggregate_pipeline.process(optimized_submaps, [np.eye(4) for _ in range(len(optimized_submaps))])
    
    o3d.visualization.draw_geometries([
        voxel_slam.aggregate_map(aggregate_output.optimized_clouds, aggregate_output.optimized_poses)
    ])
    
    # """Optimization on new poses"""
    # for voxel_key in feature_map:
    #     for pose_id, voxel_points in feature_map[voxel_key].items():
    #         pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(voxel_points.points))
    #         pcd.transform(opt_poses[pose_id])
    #         feature_map[voxel_key][pose_id].points = np.asarray(pcd.points)
    
    # opt_poses_new, _, _ = voxel_slam.EFBackend(feature_map, len(optimized_submaps)).get_optimized_poses(1000, verbose=True)

    # o3d.visualization.draw_geometries([
    #     voxel_slam.aggregate_map(aggregate_output.optimized_clouds, opt_poses_new)
    # ])

In [56]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=250, number_of_clouds=150, center_distance_threshold=3)

In [57]:
o3d.visualization.draw_geometries([
    voxel_slam.aggregate_map(clouds, poses, enable_color=True)
])

In [58]:
break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_threshold=0.125)

Submap 0-5:
adaptive_iterations = 5
FGraph initial error: 338.0408389331266
Iteratios to converge: 50
Chi2: 309.70869690922234
====
Submap 5-10:
adaptive_iterations = 5
FGraph initial error: 25.856990531144163
Iteratios to converge: 15
Chi2: 23.52886141002608
====
Submap 10-15:
adaptive_iterations = 6
FGraph initial error: 6.360507371165847
Iteratios to converge: 8
Chi2: 5.659103737729409
====
Submap 15-20:
adaptive_iterations = 5
FGraph initial error: 17.213874749473643
Iteratios to converge: 6
Chi2: 10.881492038853827
====
Submap 20-25:
adaptive_iterations = 6
FGraph initial error: 8.122822200470718
Iteratios to converge: 2
Chi2: 7.9801970992903986
====
Submap 25-30:
adaptive_iterations = 5
FGraph initial error: 95.32232273959949
Iteratios to converge: 8
Chi2: 94.63433703814351
====
Submap 30-35:
adaptive_iterations = 5
FGraph initial error: 54.33176425433832
Iteratios to converge: 21
Chi2: 53.77632118840311
====
Submap 35-40:
adaptive_iterations = 6
FGraph initial error: 92.01997889